In [1]:
!pip install mediapipe opencv-python


In [1]:
import ipywidgets as widgets
from IPython.display import display

camera_active = True

stop_button = widgets.Button(
    description='Stop Camera',
    button_style='danger',
    icon='stop'
)

def on_stop_button_clicked(b):
    global camera_active
    camera_active = False
    print("Camera stopped. Run the cell again to restart.")

stop_button.on_click(on_stop_button_clicked)
display(stop_button)


Button(button_style='danger', description='Stop Camera', icon='stop', style=ButtonStyle())

In [3]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def draw_rounded_rectangle(img, top_left, bottom_right, radius, color, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right
    cv2.rectangle(img, (x1 + radius, y1), (x2 - radius, y1 + radius), color, thickness)
    cv2.rectangle(img, (x1, y1 + radius), (x2, y2 - radius), color, thickness)
    cv2.rectangle(img, (x1 + radius, y2 - radius), (x2 - radius, y2), color, thickness)
    cv2.circle(img, (x1 + radius, y1 + radius), radius, color, thickness)
    cv2.circle(img, (x2 - radius, y1 + radius), radius, color, thickness)
    cv2.circle(img, (x1 + radius, y2 - radius), radius, color, thickness)
    cv2.circle(img, (x2 - radius, y2 - radius), radius, color, thickness)

cap = cv2.VideoCapture(0)

# Exercise parameters
counter = 0
stage = None
feedback = ""
baseline_set = False
calibration_frames = 0
calibration_sum = 0
cow_threshold = 0.03
cat_threshold = 0.02
current_phase = None
phase_history = []

# UI parameters
panel_width = 400
panel_height = 100
instruction_panel_height = 160
corner_radius = 20
bg_color = (30, 30, 30)
text_color = (255, 255, 255)
accent_color = (0, 200, 255)
progress_color = (0, 255, 200)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        h, w, _ = image.shape
        
        # ========== UI ELEMENTS ========== #
        # Instruction Panel (Top)
        top_panel_x = (w - panel_width) // 2
        top_panel_y = 20
        instruction_overlay = image.copy()
        draw_rounded_rectangle(
            instruction_overlay,
            (top_panel_x, top_panel_y),
            (top_panel_x + panel_width, top_panel_y + instruction_panel_height),
            corner_radius,
            bg_color,
            -1
        )
        alpha = 0.8
        cv2.addWeighted(instruction_overlay, alpha, image, 1 - alpha, 0, image)
        
        # Instruction Text
        cv2.putText(image, "CAT-COW STRETCH", 
                   (top_panel_x + panel_width//2 - 100, top_panel_y + 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, accent_color, 1, cv2.LINE_AA)
        instructions = [
            "1. Start on hands and knees or sit upright",
            "2. ARCH BACK (Cow): Lift head/tailbone up",
            "3. ROUND SPINE (Cat): Tuck chin/pelvis in",
            "4. Move slowly through full range",
            "5. Complete cycles for reps"
        ]
        line_spacing = 22
        start_y = top_panel_y + 60
        for i, instr in enumerate(instructions):
            y_pos = start_y + (i * line_spacing)
            cv2.putText(image, instr,
                       (top_panel_x + 25, y_pos),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.45, text_color, 1, cv2.LINE_AA)

        # Counter Panel (Bottom)
        bottom_panel_x = (w - panel_width) // 2
        bottom_panel_y = h - panel_height - 20
        counter_overlay = image.copy()
        draw_rounded_rectangle(
            counter_overlay,
            (bottom_panel_x, bottom_panel_y),
            (bottom_panel_x + panel_width, bottom_panel_y + panel_height),
            corner_radius,
            bg_color,
            -1
        )
        cv2.addWeighted(counter_overlay, alpha, image, 1 - alpha, 0, image)
        
        # Counter Display
        cv2.putText(image, "REPS COMPLETED",
                   (bottom_panel_x + panel_width//2 - 90, bottom_panel_y + 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                   (bottom_panel_x + panel_width//2 - (15 if counter < 10 else 25), bottom_panel_y + 70),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, text_color, 2, cv2.LINE_AA)
        
        # Feedback Text
        if feedback:
            cv2.putText(image, feedback,
                       (bottom_panel_x + 20, bottom_panel_y + 95),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, accent_color, 1, cv2.LINE_AA)

        # ========== EXERCISE LOGIC ========== #
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Key landmarks
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            
            # Mid-spine approximation
            mid_spine_x = (left_shoulder.x + right_shoulder.x + left_hip.x + right_hip.x) / 4
            mid_spine_y = (left_shoulder.y + right_shoulder.y + left_hip.y + right_hip.y) / 4
            
            # Spinal position calculation
            shoulder_avg_y = (left_shoulder.y + right_shoulder.y) / 2
            hip_avg_y = (left_hip.y + right_hip.y) / 2
            spine_position = mid_spine_y - ((shoulder_avg_y + hip_avg_y) / 2)

            # Calibration
            if not baseline_set:
                if calibration_frames < 30:
                    calibration_sum += spine_position
                    calibration_frames += 1
                    feedback = f"Calibrating... {calibration_frames}/30"
                else:
                    baseline = calibration_sum / calibration_frames
                    baseline_set = True
                    feedback = "Ready. Begin Cat-Cow!"
            else:
                # Phase detection
                movement = spine_position - baseline
                if movement < -cow_threshold:
                    current_phase = 'Cow'
                elif movement > cat_threshold:
                    current_phase = 'Cat'
                else:
                    current_phase = 'Neutral'
                
                # Update phase history
                phase_history.append(current_phase)
                if len(phase_history) > 5:
                    phase_history.pop(0)
                
                # Rep counting
                if len(phase_history) >= 4:
                    if (phase_history[-4] == 'Cow' and
                        phase_history[-3] == 'Neutral' and
                        phase_history[-2] == 'Cat' and
                        phase_history[-1] == 'Neutral'):
                        counter += 1
                        feedback = "Rep Completed!"
                        phase_history.clear()

        except Exception as e:
            pass

        # ========== VISUALIZATION ========== #
        if results.pose_landmarks:
            landmarks_overlay = image.copy()
            
            # Draw spine curve
            try:
                shoulder_mid = (int((left_shoulder.x + right_shoulder.x)/2 * w),
                               int((left_shoulder.y + right_shoulder.y)/2 * h))
                hip_mid = (int((left_hip.x + right_hip.x)/2 * w),
                          int((left_hip.y + right_hip.y)/2 * h))
                mid_spine_pt = (int(mid_spine_x * w), int(mid_spine_y * h))
                
                curve_points = np.array([shoulder_mid, mid_spine_pt, hip_mid], dtype=np.int32)
                cv2.polylines(landmarks_overlay, [curve_points], False, progress_color, 2)
                
                landmarks_alpha = 0.3
                cv2.addWeighted(landmarks_overlay, landmarks_alpha, image, 1 - landmarks_alpha, 0, image)
            except:
                pass

        cv2.imshow('Cat-Cow Tracker', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
